In [ ]:
# Install EasyOCR
!pip3 install easyocr


In [ ]:
import easyocr
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Initialize the OCR reader
reader = easyocr.Reader(['en'])  # This needs to be run only once to load the model

# Define a simple function to perform OCR on an image
def easy_ocr(image_path):
    try:
        # Read image
        result = reader.readtext(image_path)
        
        # Display the image
        img = Image.open(image_path)
        plt.figure(figsize=(10, 10))
        plt.imshow(np.array(img))
        plt.axis('off')
        plt.title('Input Image')
        plt.show()
        
        # Extract and return full text
        full_text = ""
        for detection in result:
            text = detection[1]  # The text is the second element in each detection
            full_text += text + " "
            
        return full_text
    except Exception as e:
        return f"Error processing image: {str(e)}"

# Path to your image
image_path = ''

# Run OCR and print the extracted text
extracted_text = easy_ocr(image_path)
print("Extracted text from image:")
print("-" * 40)
print(extracted_text)